In [1]:
# !python -m nltk.downloader punkt

In [2]:
# !git clone https://github.com/patil-suraj/question_generation.git

In [1]:
import os
import pandas as pd
from os import listdir
from os.path import isfile, join
import re

In [2]:
print(os.getcwd())

/mnt/d/MASTER2_NLP/Software project/env


In [3]:
os.chdir("../")
print(os.getcwd())

/mnt/d/MASTER2_NLP/Software project


In [4]:
from pipeline_fr import pipeline_fr

/mnt/d/MASTER2_NLP/Software project/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data_dir = "Datasets/CASS-dataset/Freemium_cass_global_20220417-170000/cass/global/civile"

# Run this cell and the next one just one time to generate the 'cases_after_2000.txt' file

I uploaded this file on git so normally you should find it there

Filter and select just the cases after year 2000 

In [6]:
import re
"""
files_after_2000 = []
i=0
for path, _, files in os.walk(data_dir):
    for name in files:
        if i<1000:
            with open(os.path.join(path, name), 'r', encoding='utf-8', errors='ignore') as myfile:
                full_text = myfile.read()
                date = None
                
                if re.search('<META_JURI>.*</META_JURI>', full_text, re.DOTALL):
                    
                    search = re.search('(?<=<DATE_DEC>).*?(?=</DATE_DEC>)', full_text, re.DOTALL)
                    date = search.group(0)
                    y, m, d = date.split("-")
                    if (int(y) >= 2000):
                        files_after_2000.append(name)
                        
"""


'\nfiles_after_2000 = []\ni=0\nfor path, _, files in os.walk(data_dir):\n    for name in files:\n        if i<1000:\n            with open(os.path.join(path, name), \'r\', encoding=\'utf-8\', errors=\'ignore\') as myfile:\n                full_text = myfile.read()\n                date = None\n                \n                if re.search(\'<META_JURI>.*</META_JURI>\', full_text, re.DOTALL):\n                    \n                    search = re.search(\'(?<=<DATE_DEC>).*?(?=</DATE_DEC>)\', full_text, re.DOTALL)\n                    date = search.group(0)\n                    y, m, d = date.split("-")\n                    if (int(y) >= 2000):\n                        files_after_2000.append(name)\n                        \n'

Write all these file names in a .txt file

In [7]:
"""
text_file = open("cases_after_2000.txt", "wt")
for case in files_after_2000:
    case = case.replace(".xml", "")
    text_file.write(case)
    text_file.write("\n")
    
text_file.close()

"""

'\ntext_file = open("cases_after_2000.txt", "wt")\nfor case in files_after_2000:\n    case = case.replace(".xml", "")\n    text_file.write(case)\n    text_file.write("\n")\n    \ntext_file.close()\n\n'

Dont't forget to change the path if needed:

In [8]:
path_cases_file = open("cases_after_2000.txt", "r")
cases_file = path_cases_file.read()
str_files = cases_file.split("\n")

In [9]:
print(len(str_files))

17314


In [10]:
df = pd.DataFrame(columns = ["id_file", "decision", "resume"])
df

,id_file,decision,resume


In [11]:
def check_file(file):
    file = file.replace(".story", "")
    if file in str_files:
        return 1
    
    return 0

Check if a .story file is after year 2000,  read it and save the content in a vector

In [12]:
path = "env/cleaned_files_civile"
content_file = []
file_content_assoc={}
for file in os.listdir(path):
    if check_file(file):
        with open(os.path.join(path, file), "r") as f:
            content = f.read()
            content_file.append(content)
            file_content_assoc[file] = content
        

Apparently in the "cleaned_files_civile" directory there were less files after year 2000:
- in the original dataset: 17314
- from the generate .story files: 16256

Maybe there was a bug during their generation and some of them were lost. It is not such a problem, it is just informative

In [13]:
len(file_content_assoc)

16256

Split each file by "@highlight" and save the decision and resume in differet variables

In [15]:
decision, resume = [], []
for f in content_file:
    d, r = f.split("@highlight")
    decision.append(d)
    resume.append(r)

Delete special characters from decision and resume (for example "\n")

In [36]:
def delete_special_characters(d):
    new_d = []
    for item in d:
        item = item.replace("\n", "")
        item = item.replace("' ", "'")
        item = item.replace('\\' , '' )
        #item = item.replace('. ' , '.' )
        new_d.append(item)
    return new_d

In [37]:
decision = delete_special_characters(decision)
resume = delete_special_characters(resume)
print(len(decision), len(resume))

16256 16256


In [38]:
df_files = pd.DataFrame(list(zip( list(file_content_assoc.keys()), decision, resume)), columns = ["id_file", "decision", "resume"])
df_files

,id_file,decision,resume
0,JURITEXT000007040187.story,donne acte a l'office public d'amenagement et ...,les epoux demeurent cotitulaires du bail jusqu...
1,JURITEXT000007040188.story,"sur le moyen unique :attendu , selon l'arret a...",la cour d'appel qui releve qu'une meme lettre ...
2,JURITEXT000007040210.story,sur le moyen unique :attendu que par acte du 2...,l'article l. 121 - 10 du code des assurances s...
3,JURITEXT000007040335.story,arret n 2sur le moyen unique :vu les articles ...,le delai d'un mois imparti pour elever une con...
4,JURITEXT000007040336.story,"attendu , selon l'arret confirmatif attaque , ...",1ne constitue pas une contestation de la saisi...
...,...,...,...
16251,JURITEXT000045470245.story,"la cour de cassation , deuxieme chambre civile...",n'est pas une clause penale au sens de l'artic...
16252,JURITEXT000045470247.story,"la cour de cassation , deuxieme chambre civile...",c'est dans l'exercice de son pouvoir souverain...
16253,JURITEXT000045470249.story,"la cour de cassation , deuxieme chambre civile...",si un accident de la circulation trouve sa cau...
16254,JURITEXT000045545615.story,"la cour de cassation , deuxieme chambre civile...",il resulte de l'article 1er de la loi du 5 jui...


In [40]:
nlp_fr = pipeline_fr("question-generation")

In [44]:
vector = []
data_context_questions = {}
for ind in df_files.index:
    data_context_questions = {'documentID':'', 'items':[]}
    data_context_questions['documentID'] = df_files['id_file'][ind]
    
    # set ind = 16256 to generate questions for all files
    if(ind < 50):
        sentences = df_files['resume'][ind]
        sent = re.split(';| \.', sentences)
        sent.remove(' ')
        item_sent_questions = {}
        for s in sent:
            item_sent_questions = {}
            
            # sometimes some sentense starts with a digit like: "1l'arret"
            # I eliminated the digit cause it makes the entire word unrecognizable
            s = re.sub(r"\d", "", s, 1)
            item_sent_questions = {"context": s, "questions":[]}
            try:
                qs = nlp_fr(s)
                for q in qs:
                    dict_q = {"question": q['question']}
                    item_sent_questions["questions"].append(dict_q)
            except:
                print("Not able to generate question -- en error occured")
            data_context_questions["items"].append(item_sent_questions)
        vector.append(data_context_questions)    
                

[{'answer': 'loyers', 'question': 'Les epoux sont tenus solidairement au paiement de quoi?'}]
[{'documentID': 'JURITEXT000007040187.story', 'items': [{'context': "les epoux demeurent cotitulaires du bail jusqu'a la transcription du jugement de divorce en marge des registres de l'etat civil et sont des lors tenus solidairement au paiement des loyers , nonobstant le fait que la separation des conjoints a ete autorisee par le juge et portee a la connaissance du bailleur", 'questions': [{'question': 'Les epoux sont tenus solidairement au paiement de quoi?'}]}]}]
[{'answer': '23 decembre 1986', 'question': "quelle date la proposition d'un nouveau contrat a-t-elle été faite?"}, {'answer': 'epoux colocataires', 'question': "qui l'avis de réception a-t-il été signé?"}]
[{'documentID': 'JURITEXT000007040187.story', 'items': [{'context': "les epoux demeurent cotitulaires du bail jusqu'a la transcription du jugement de divorce en marge des registres de l'etat civil et sont des lors tenus solidair

[{'answer': '15 septembre 1991', 'question': "Quand la cour d'appel a-t-elle décidé que le conge d'un local a usage professionnel consenti pour 6"}, {'answer': '6 mois', 'question': "Combien de temps faut-il pour que le locataire d'un local ait l'intention de quitter les"}]
[{'documentID': 'JURITEXT000007040187.story', 'items': [{'context': "les epoux demeurent cotitulaires du bail jusqu'a la transcription du jugement de divorce en marge des registres de l'etat civil et sont des lors tenus solidairement au paiement des loyers , nonobstant le fait que la separation des conjoints a ete autorisee par le juge et portee a la connaissance du bailleur", 'questions': [{'question': 'Les epoux sont tenus solidairement au paiement de quoi?'}]}]}, {'documentID': 'JURITEXT000007040188.story', 'items': [{'context': "la cour d'appel qui releve qu'une meme lettre recommandee avait ete adressee aux epoux colocataires et que l'avis de reception avait ete signe par chacun d'eux en deduit exactement que l

[{'answer': 'ne tire pas les consequences legales de ses constatations', 'question': "Qu'est-ce que la cour d'appel a déclaré que les desordres de coulure ne sont pas ines"}]
[{'documentID': 'JURITEXT000007040187.story', 'items': [{'context': "les epoux demeurent cotitulaires du bail jusqu'a la transcription du jugement de divorce en marge des registres de l'etat civil et sont des lors tenus solidairement au paiement des loyers , nonobstant le fait que la separation des conjoints a ete autorisee par le juge et portee a la connaissance du bailleur", 'questions': [{'question': 'Les epoux sont tenus solidairement au paiement de quoi?'}]}]}, {'documentID': 'JURITEXT000007040188.story', 'items': [{'context': "la cour d'appel qui releve qu'une meme lettre recommandee avait ete adressee aux epoux colocataires et que l'avis de reception avait ete signe par chacun d'eux en deduit exactement que la notification de la proposition d'un nouveau contrat en application de l'article 1 de la loi du 23 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [46]:
import json 
# Serializing json  
json_object = json.dumps(vector) 
print(json_object)

[{"documentID": "JURITEXT000007040187.story", "items": [{"context": "les epoux demeurent cotitulaires du bail jusqu'a la transcription du jugement de divorce en marge des registres de l'etat civil et sont des lors tenus solidairement au paiement des loyers , nonobstant le fait que la separation des conjoints a ete autorisee par le juge et portee a la connaissance du bailleur", "questions": [{"question": "Les epoux sont tenus solidairement au paiement de quoi?"}]}]}, {"documentID": "JURITEXT000007040188.story", "items": [{"context": "la cour d'appel qui releve qu'une meme lettre recommandee avait ete adressee aux epoux colocataires et que l'avis de reception avait ete signe par chacun d'eux en deduit exactement que la notification de la proposition d'un nouveau contrat en application de l'article 1 de la loi du 23 decembre 1986 avait ete regulierement effectuee", "questions": [{"question": "quelle date la proposition d'un nouveau contrat a-t-elle \u00e9t\u00e9 faite?"}, {"question": "qu

Save the json object in a file

In [50]:
with open("sample.json", "w", encoding='utf8') as outfile:
    json.dump(vector, outfile, ensure_ascii=False, indent = 4)